In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
254,AFG,Asia,Afghanistan,2020-09-10,38544.0,24.0,43.000,1420.0,2.0,1.571,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
440,ALB,Europe,Albania,2020-09-10,10704.0,151.0,139.429,322.0,1.0,3.714,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
695,DZA,Africa,Algeria,2020-09-10,47216.0,278.0,294.000,1581.0,10.0,8.000,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-09,France,339461.0,0.0,30803.0,0.0,0.0,0.0
0,2020-09-09,China,90108.0,0.0,4739.0,0.0,0.0,0.0
0,2020-09-09,Italy,281198.0,0.0,35593.0,0.0,0.0,0.0
0,2020-09-09,Spain,549374.0,0.0,29681.0,0.0,0.0,0.0
0,2020-09-09,United States,6415466.0,0.0,190564.0,0.0,0.0,0.0
0,2020-09-09,World,28207519.0,0.0,907034.0,0.0,0.0,0.0
0,2020-09-09,United Kingdom,354996.0,0.0,41622.0,0.0,0.0,0.0
0,2020-09-09,Germany,254834.0,0.0,9347.0,0.0,0.0,0.0
0,2020-09-09,Iran,394810.0,0.0,22776.0,0.0,0.0,0.0
0,2020-09-09,Turkey,284616.0,0.0,6828.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-10,France,339461.0,344101.0,30803.0,30794.0,-4640.0,9.0
0,2020-09-10,China,90108.0,90100.0,4739.0,4733.0,8.0,6.0
0,2020-09-10,Italy,281198.0,281853.0,35593.0,35577.0,-655.0,16.0
0,2020-09-09,Spain,549374.0,543379.0,29681.0,29628.0,5995.0,53.0
0,2020-09-10,United States,6415466.0,6359720.0,190564.0,190815.0,55746.0,-251.0
0,2020-09-10,World,28207519.0,27891329.0,907034.0,903991.0,316190.0,3043.0
0,2020-09-10,United Kingdom,354996.0,355219.0,41622.0,41594.0,-223.0,28.0
0,2020-09-10,Germany,254834.0,255366.0,9347.0,9341.0,-532.0,6.0
0,2020-09-10,Iran,394810.0,393425.0,22776.0,22669.0,1385.0,107.0
0,2020-09-10,Turkey,284616.0,284943.0,6828.0,6837.0,-327.0,-9.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")